# Iris분류

#### Logistic Regression

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()

In [3]:
# data, target, target_name, feature_names, DESCR
data = iris.data
label = iris.target
columns = iris.feature_names

df = pd.DataFrame(data, columns = columns)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size = 0.2,
                                                    shuffle = True, stratify
                                                    = label,
                                                    random_state = 2019)

- 모델 객체 생성

In [5]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(verbose = 1)

- 모델 학습

In [6]:
lr.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(verbose=1)

- 학습한 결과를 저장

In [9]:
import joblib
joblib.dump(lr, 'model/iris_lr.pkl')

['model/iris_lr.pkl']

In [10]:
from sklearn.svm import SVC

In [18]:
svm = SVC()

In [19]:
svm.fit(x_train, y_train)

SVC()

In [20]:
joblib.dump(svm, 'model/iris_svm.pkl')

['model/iris_svm.pkl']

### Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
dt = DecisionTreeClassifier()

In [22]:
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [23]:
joblib.dump(dt, 'model/iris_dt.pkl')

['model/iris_dt.pkl']

#### 저장된 모델로 결과 예측

In [40]:
model_lr = joblib.load('model/iris_lr.pkl')
model_svm = joblib.load('model/iris_svm.pkl')
model_dt = joblib.load('model/iris_dt.pkl')

In [41]:
y_pred_lr = model_lr.predict(x_test)
y_pred_svm = model_svm.predict(x_test)
y_pred_dt = model_dt.predict(x_test)

In [42]:
from sklearn.metrics import accuracy_score

In [47]:
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(acc_lr, acc_svm, acc_dt)

0.9666666666666667 0.9666666666666667 0.9666666666666667


In [48]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [49]:
test_data = np.array([6.7, 3.0, 5.2, 2.3]).reshape(1, 4)

In [50]:
index_lr = model_lr.predict(test_data)[0]
index_svm = model_svm.predict(test_data)[0]
index_dt = model_dt.predict(test_data)[0]

In [52]:
sp_names = ['Setosa', 'Versicolor', 'Virginica']
print(sp_names[index_lr], sp_names[index_svm], sp_names[index_dt])

Virginica Virginica Virginica


In [60]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [71]:
model = Sequential([
    Dense(12, input_shape = (4,), activation = 'relu'),
    Dense(8, input_shape = (4,), activation = 'relu'),
    Dense(3, activation = 'softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 12)                60        
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [73]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath = 'model/iris_deep.hdf5',
                               monitor = 'val_loss',
                               verbose = 1, save_best_only = True)

In [74]:
Y_encoded = keras.utils.to_categorical(y_train)
Y_encoded[0], Y_encoded[50], Y_encoded[100]

(array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32))

In [78]:
model.fit(x_train, Y_encoded, epochs = 100, batch_size = 30, verbose = 2,
           validation_split = 0.2, callbacks = [checkpointer])

Train on 96 samples, validate on 24 samples
Epoch 1/100
 - 0s - loss: 1.8728 - accuracy: 0.3438 - val_loss: 1.9124 - val_accuracy: 0.2917

Epoch 00001: val_loss improved from inf to 1.91241, saving model to model/iris_deep.hdf5
Epoch 2/100
 - 0s - loss: 1.8023 - accuracy: 0.3438 - val_loss: 1.8364 - val_accuracy: 0.2917

Epoch 00002: val_loss improved from 1.91241 to 1.83636, saving model to model/iris_deep.hdf5
Epoch 3/100
 - 0s - loss: 1.7305 - accuracy: 0.3438 - val_loss: 1.7638 - val_accuracy: 0.2917

Epoch 00003: val_loss improved from 1.83636 to 1.76384, saving model to model/iris_deep.hdf5
Epoch 4/100
 - 0s - loss: 1.6633 - accuracy: 0.3438 - val_loss: 1.6929 - val_accuracy: 0.2917

Epoch 00004: val_loss improved from 1.76384 to 1.69291, saving model to model/iris_deep.hdf5
Epoch 5/100
 - 0s - loss: 1.5989 - accuracy: 0.3438 - val_loss: 1.6257 - val_accuracy: 0.2917

Epoch 00005: val_loss improved from 1.69291 to 1.62567, saving model to model/iris_deep.hdf5
Epoch 6/100
 - 0s - 


Epoch 00044: val_loss improved from 0.70529 to 0.69520, saving model to model/iris_deep.hdf5
Epoch 45/100
 - 0s - loss: 0.6788 - accuracy: 0.7083 - val_loss: 0.6868 - val_accuracy: 0.7917

Epoch 00045: val_loss improved from 0.69520 to 0.68677, saving model to model/iris_deep.hdf5
Epoch 46/100
 - 0s - loss: 0.6693 - accuracy: 0.7083 - val_loss: 0.6792 - val_accuracy: 0.8333

Epoch 00046: val_loss improved from 0.68677 to 0.67917, saving model to model/iris_deep.hdf5
Epoch 47/100
 - 0s - loss: 0.6603 - accuracy: 0.8125 - val_loss: 0.6726 - val_accuracy: 0.8333

Epoch 00047: val_loss improved from 0.67917 to 0.67260, saving model to model/iris_deep.hdf5
Epoch 48/100
 - 0s - loss: 0.6522 - accuracy: 0.8854 - val_loss: 0.6669 - val_accuracy: 0.8750

Epoch 00048: val_loss improved from 0.67260 to 0.66693, saving model to model/iris_deep.hdf5
Epoch 49/100
 - 0s - loss: 0.6439 - accuracy: 0.8958 - val_loss: 0.6596 - val_accuracy: 1.0000

Epoch 00049: val_loss improved from 0.66693 to 0.65956

Epoch 88/100
 - 0s - loss: 0.4480 - accuracy: 0.9271 - val_loss: 0.4548 - val_accuracy: 1.0000

Epoch 00088: val_loss improved from 0.45769 to 0.45482, saving model to model/iris_deep.hdf5
Epoch 89/100
 - 0s - loss: 0.4448 - accuracy: 0.9375 - val_loss: 0.4529 - val_accuracy: 1.0000

Epoch 00089: val_loss improved from 0.45482 to 0.45287, saving model to model/iris_deep.hdf5
Epoch 90/100
 - 0s - loss: 0.4421 - accuracy: 0.9375 - val_loss: 0.4509 - val_accuracy: 1.0000

Epoch 00090: val_loss improved from 0.45287 to 0.45094, saving model to model/iris_deep.hdf5
Epoch 91/100
 - 0s - loss: 0.4388 - accuracy: 0.9479 - val_loss: 0.4474 - val_accuracy: 1.0000

Epoch 00091: val_loss improved from 0.45094 to 0.44736, saving model to model/iris_deep.hdf5
Epoch 92/100
 - 0s - loss: 0.4358 - accuracy: 0.9479 - val_loss: 0.4426 - val_accuracy: 1.0000

Epoch 00092: val_loss improved from 0.44736 to 0.44265, saving model to model/iris_deep.hdf5
Epoch 93/100
 - 0s - loss: 0.4327 - accuracy: 0.9479 - 

In [79]:
from keras.models import load_model
model_deep = load_model('model/iris_deep.hdf5')

In [81]:
model.evaluate(x_test, keras.utils.to_categorical(y_test))[1]

30/30 [==============================] - 0s 33us/step


0.9333333373069763

In [86]:
model_deep.predict_classes(test_data)[0]

2

In [83]:
model_deep.predict(test_data)

array([[0.01212918, 0.38598475, 0.601886  ]], dtype=float32)

In [84]:
np.argmax(model_deep.predict(test_data)[0])

2